# Install and import packages
--------

In [17]:
# install/import quantum gym environments
#!pip install git+https://github.com/qdevpsi3/quantum-arch-search.git

# install/import stable baselines 3
#!pip install stable_baselines3

In [18]:
import gym
import numpy as np
import torch.optim as optim
from stable_baselines3 import A2C, PPO
from stable_baselines3.common.evaluation import evaluate_policy
import qas_gym

# Basic Environment
------
Create your gym environment :

In [19]:
# Parameters 
env_name = 'BasicTwoQubit-v0' # 两比特环境-无噪声
fidelity_threshold = 0.9
reward_penalty = 0.1
max_timesteps = 20
target = np.asarray([0.70710678+0.j,0. +0.j,0. +0.j, 0.70710678+0.j])
# Environment
env = gym.make(env_name, target = target,
               fidelity_threshold=fidelity_threshold,
               reward_penalty=reward_penalty,
               max_timesteps=max_timesteps)

Diplay the action gates : 

In [20]:
for idx, gate in enumerate(env.action_gates):
    print('Action({:02d}) --> {}'.format(idx, gate))

Action(00) -->  *** gate info *** 
 * gate name : Z-rotation
 * target    : 
 0 : commute     Z 
 * control   : 
 * Pauli     : no
 * Clifford  : no
 * Gaussian  : no
 * Parametric: no
 * Diagonal  : yes

Action(01) -->  *** gate info *** 
 * gate name : X
 * target    : 
 0 : commute X     
 * control   : 
 * Pauli     : yes
 * Clifford  : yes
 * Gaussian  : no
 * Parametric: no
 * Diagonal  : no

Action(02) -->  *** gate info *** 
 * gate name : Y
 * target    : 
 0 : commute   Y   
 * control   : 
 * Pauli     : yes
 * Clifford  : yes
 * Gaussian  : no
 * Parametric: no
 * Diagonal  : no

Action(03) -->  *** gate info *** 
 * gate name : Z
 * target    : 
 0 : commute     Z 
 * control   : 
 * Pauli     : yes
 * Clifford  : yes
 * Gaussian  : yes
 * Parametric: no
 * Diagonal  : yes

Action(04) -->  *** gate info *** 
 * gate name : H
 * target    : 
 0 : commute       
 * control   : 
 * Pauli     : no
 * Clifford  : yes
 * Gaussian  : no
 * Parametric: no
 * Diagonal  : no

Action

Diplay the state observables : 

In [21]:
for idx, observable in enumerate(env.state_observables):
    print('State({:02d}) --> {}'.format(idx, observable))

# for idx, observable in enumerate(env.pauli_observables):
#     print('State({:02d}) --> {}'.format(idx, observable))   

State(00) --> 1 X 0
State(01) --> 1 Y 0
State(02) --> 1 Z 0
State(03) --> 1 X 1
State(04) --> 1 Y 1
State(05) --> 1 Z 1


In [22]:
from qulacs import QuantumState, QuantumCircuit, QuantumCircuitSimulator, Observable
n = 2
state = QuantumState(n)

circuit = QuantumCircuit(n)
for i in range(n):
   circuit.add_H_gate(i)

sim = QuantumCircuitSimulator(circuit, state)

sim.initialize_state(0)

print("gate_count: ", sim.get_gate_count())

sim.simulate()

print(state)


gate_count:  2
 *** Quantum State ***
 * Qubit Count : 2
 * Dimension   : 4
 * State vector : 
(0.5,0)
(0.5,0)
(0.5,0)
(0.5,0)



In [23]:
observable = Observable(2)
observable.add_operator(1.0, "Z 1")
# print("expectation_value: ", sim.get_expectation_value(observable))

state = QuantumState(2)
state.load([.65328148-0.27059805j, 0.-0.j, 0.65328148-0.27059805j, 0.-0.j])

observable.get_expectation_value(state)

0.0

In [24]:
observable = Observable(2)
observable.add_operator(1.0, "Z 0")
obs = observable.get_matrix()
print(obs.todense())

[[ 1.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j -1.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  1.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j -1.+0.j]]


In [25]:
observable.get_term(0)

In [26]:
observable.get_state_dim()

4

In [27]:
a = observable.get_term(0)
a.get_pauli_string()

'Z 0'

In [28]:
a.get_pauli_id_list()

[3]

In [29]:
# from cirq.work.observable_measurement import measure_observables, RepetitionsStoppingCriteria
# import cirq
# # circuit = env._get_cirq(maybe_add_noise=True)
# # sample_pauli_obs = env.pauli_observables[0]
# circuit = cirq.Circuit(cirq.I(qubit) for qubit in env.qubits)
# # result = measure_observables(
# #                 circuit, env.pauli_observables, cirq.Simulator(), stopping_criteria=RepetitionsStoppingCriteria(100))
# K=10
# # sample_pauli_idx = np.random.choice(len(env.pauli_observables), K)
# sample_pauli = [env.pauli_observables[i] for i in np.random.choice(len(env.pauli_observables), K)]
# result = measure_observables(
#     circuit, sample_pauli, cirq.Simulator(), stopping_criteria=RepetitionsStoppingCriteria(100))
  
# # print(result[15].mean, result[15].variance)

In [30]:
# for observable in env.pauli_observables:
#    print(type(observable))
#    for qubit, pauli in observable.items():
#       print(qubit, pauli)

# print(result[1].mean, result[1].variance)

# A2C Agent
------

In [31]:
# # Parameters
# gamma = 0.99
# learning_rate = 0.0001
# policy_kwargs = dict(optimizer_class=optim.Adam)

# # Agent
# a2c_model = A2C("MlpPolicy",
#                 env,
#                 gamma=gamma,
#                 learning_rate=learning_rate,
#                 policy_kwargs=policy_kwargs,
#                 tensorboard_log='logs/')

C:\Users\Mac\AppData\Roaming\Python\Python311\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


In [32]:
# a2c_model.learn(total_timesteps=20000)

C:\Users\Mac\AppData\Roaming\Python\Python311\site-packages\gym\utils\passive_env_checker.py:174: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed a `seed` instead of using `Env.seed` for resetting the environment random number generator.
  logger.warn(
C:\Users\Mac\AppData\Roaming\Python\Python311\site-packages\gym\utils\passive_env_checker.py:187: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed `options` to allow the environment initialisation to be passed additional information.
  logger.warn(
C:\Users\Mac\AppData\Roaming\Python\Python311\site-packages\gym\utils\passive_env_checker.py:195: UserWarning: WARN: The result returned by `env.reset()` was not a tuple of the form `(obs, info)`, where `obs` is a observation and `info` is a dictionary containing additional information. Actual type: `<class 'numpy.ndarray'>`
  logger.warn(
C:\Users\Mac\AppData\Roaming\Python\Python311\site-packages\gym\utils\passive_env_check

[0.-0.j 0.+1.j 0.-0.j 0.+0.j]
[0.-0.j         0.+0.70710678j 0.+0.j         0.+0.70710678j]
[0.+0.j 0.+1.j 0.-0.j 0.+0.j]
[ 0.+0.j -0.-1.j  0.-0.j -0.-0.j]
[0.+0.j 0.+1.j 0.-0.j 0.+0.j]
[1.-0.j 0.+0.j 0.-0.j 0.+0.j]
[-0.+0.j  0.-0.j  0.+1.j  0.+0.j]
[ 0.+0.70710678j  0.+0.j         -0.-0.70710678j  0.-0.j        ]
[ 0.+0.70710678j  0.-0.j         -0.-0.70710678j  0.+0.j        ]
[ 0.+0.70710678j  0.-0.j          0.+0.j         -0.-0.70710678j]
[ 0.+0.70710678j  0.-0.j         -0.-0.70710678j  0.+0.j        ]
[0.+0.j 0.+0.j 0.+1.j 0.-0.j]
[0.+0.j         0.+0.j         0.+0.70710678j 0.+0.70710678j]
[0.+0.j         0.+0.70710678j 0.+0.70710678j 0.+0.j        ]
[ 0.70710678-0.j  0.        -0.j  0.        +0.j -0.70710678+0.j]
[ 0.        -0.j  0.70710678-0.j -0.70710678+0.j  0.        +0.j]
[ 0.        -0.j  0.        +0.j -0.70710678+0.j  0.70710678-0.j]
[-0.70710678+0.j  0.70710678-0.j  0.        -0.j  0.        +0.j]
[ 0.+0.j -1.+0.j  0.+0.j  0.-0.j]
[ 0.+0.j  0.-0.j  0.+0.j -1.+0.j]


KeyboardInterrupt: 

In [ ]:
# # 从初始态开始，每一步用训练好的agent对当前state选择添加的门（action）
# state = env.reset()
# print(state)
# done = False
# while not done:
#     action = a2c_model.predict(state)
#     state, reward, done, info = env.step(action[0])
#     #展示当前的线路 和 state
#     # env.render()
#     print(state)

# print(info['circuit'])
# print(info['fidelity'])

In [ ]:
action

# PPO Model
------

In [33]:
# Parameters
gamma = 0.99
n_epochs = 4
clip_range = 0.2
learning_rate = 0.0001
policy_kwargs = dict(optimizer_class=optim.Adam)


# Agent
ppo_model = PPO("MlpPolicy",
                env,
                gamma=gamma,
                n_epochs=n_epochs,
                clip_range=clip_range,
                learning_rate=learning_rate,
                policy_kwargs=policy_kwargs,
                tensorboard_log='logs/')

In [34]:
ppo_model.learn(total_timesteps=200)

[ 1.+0.j  0.+0.j -0.-0.j -0.-0.j]
[0.-0.j 0.+1.j 0.+0.j 0.-0.j]
[ 1.-0.j  0.+0.j -0.+0.j  0.+0.j]
[0.+0.j 0.-0.j 0.+1.j 0.+0.j]
[0.+0.j 0.-0.j 0.+0.j 0.+1.j]
[0.+0.j         0.+0.70710678j 0.+0.j         0.-0.70710678j]
[ 0.70710678-0.j  0.        +0.j -0.70710678+0.j  0.        +0.j]
[-0.70710678+0.j  0.        +0.j  0.70710678-0.j  0.        +0.j]
[ 0.-0.j         -0.-0.70710678j  0.-0.j          0.+0.70710678j]
[-0.        +0.j  0.70710678-0.j  0.        +0.j  0.70710678-0.j]
[-0.        +0.j  0.70710678-0.j  0.        +0.j  0.70710678-0.j]
[-0.        +0.j -0.70710678+0.j  0.        +0.j -0.70710678+0.j]
[-0.        +0.j -0.70710678+0.j -0.        -0.j  0.70710678-0.j]
[-0.        +0.j -0.70710678+0.j  0.70710678-0.j -0.        -0.j]
[-0.        +0.j -0.70710678+0.j -0.70710678+0.j  0.        +0.j]
[-0.5+0.j -0.5+0.j  0.5+0.j -0.5+0.j]
[ 0.5+0.j -0.5+0.j -0.5+0.j -0.5+0.j]
[ 0.        +0.j  0.70710678+0.j -0.70710678+0.j  0.        +0.j]
[ 0.        +0.j  0.70710678+0.j  0.        

In [35]:
# 从初始态开始，每一步用训练好的agent对当前state选择添加的门（action）
state = env.reset()
done = False
while not done:
    action = ppo_model.predict(state)
    state, reward, done, info = env.step(action[0])
    #展示当前的线路 和 state
    env.render()
    print(state)
    
print(info['fidelity'])

[1.+0.j 0.+0.j 0.+0.j 0.+0.j]


C:\Users\Mac\AppData\Roaming\Python\Python311\site-packages\gym\utils\passive_env_checker.py:272: UserWarning: WARN: No render modes was declared in the environment (env.metadata['render_modes'] is None or not defined), you may have trouble when calling `.render()`.
  logger.warn(


TypeError: exceptions must derive from BaseException

In [ ]:
print(info['circuit'])
# print(info['fidelity'])

# Results
------

In [ ]:
import os
os.environ['TENSORBOARD_BINARY'] = 'c:\\users\\mac\\appdata\\roaming\\python\\python311\\site-packages\\tensorboard'
%load_ext tensorboard
%tensorboard --logdir=logs/

In [ ]:
# 从初始态开始，每一步用训练好的agent对当前state选择添加的门（action）
state = env.reset()
done = False
while not done:
    action = ppo_model.predict(state)
    state, reward, done, info = env.step(action[0])
    #展示当前的线路 和 state
    env.render()
    print(state)


In [ ]:
from qulacs import QuantumCircuit

circuit = QuantumCircuit(2)


In [ ]:
env.action_gates



In [ ]:
a = env.action_gates[5]
a.__str__()
a.get_name()

In [ ]:
a.get_control_index_list()

In [ ]:
a.get_target_index_list()

In [ ]:
import cirq
help(cirq.LineQubit)